In [ ]:
try:
    import hugsvision
except:
    !pip install -q hugsvision
    import hugsvision

print(hugsvision.__version__)


0.75.5


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!pip conda install -c conda-forge huggingface_hub



ERROR: unknown command "conda"


In [ ]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 122545 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.4.0-2ubuntu1.2_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Setting up libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Processing triggers for man-db (2.9.1-1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.6 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=446f7a9467815ca954808ebab3f1bb66b12fdfc36fcba6bcc0d37efd32ee43b2
  Stored in directory: /root/.cache/pip/wheels/3a/94/d0/6cd83c8a80a4236fd4cb2a1fd846ecf72ab1e0ac238c5951c0
Successfully built libarchive


In [ ]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
	"/content/gdrive/MyDrive/images data 4 classes",
	test_ratio   = 0.15,
	balanced     = True,
	augmentation = True,
)

Split Datasets...
Balance train dataset...
The less represented label in train as 496 occurrences
Size of train after balancing is 1984
Training Dataset Elements:  1686
+---------+---------------+-------------+---------------+-----------------------------------------+-------+
| Dataset | Heterocycles  | Homocycles  | Hydrocarbons  | Oxygen and Nitrogen containing acyclic  | Total |
+---------+---------------+-------------+---------------+-----------------------------------------+-------+
|  Train  |      416      |     428     |      416      |                   426                   | 1686  |
|  Test   |      80       |     68      |      80       |                   70                    |  298  |
+---------+---------------+-------------+---------------+-----------------------------------------+-------+


In [ ]:
huggingface_model = 'google/vit-base-patch16-224-in21k'


In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification

trainer = VisionClassifierTrainer(
	model_name   = "MyKvasirV2Model",
	train      	 = train,
	test      	 = test,
	output_dir   = "./out/",
	max_epochs   = 2,
	batch_size   = 32, # On RTX 2080 Ti
    lr 		     = 2e-5,
	fp16	     = False,
	model = ViTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label
	),
	feature_extractor = ViTFeatureExtractor.from_pretrained(
		huggingface_model,
	),
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-p

{'0': 'Heterocycles ', '1': 'Homocycles ', '2': 'Hydrocarbons ', '3': 'Oxygen and Nitrogen containing acyclic '}
{'Heterocycles ': '0', 'Homocycles ': '1', 'Hydrocarbons ': '2', 'Oxygen and Nitrogen containing acyclic ': '3'}
Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,0.631046
2,No log,0.455788


Model saved at: ./out/MYKVASIRV2MODEL/2_2023-05-27-10-24-29


In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ref, hyp)
labels = list(label2id.keys())
df_cm = pd.DataFrame(cm, index = labels, columns = labels)

plt.figure(figsize = (10,7))
#sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, fmt="")
#plt.savefig("Colab Notebooks.jpg")

<Figure size 1000x700 with 0 Axes>

In [ ]:
import os.path
import json

from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

feature_extractor_path = "/content/out/MYKVASIRV2MODEL/15_2023-05-27-10-01-47/feature_extractor"
model_path = "/content/out/MYKVASIRV2MODEL/15_2023-05-27-10-01-47/model"
img  = "/content/gdrive/MyDrive/images data 4 classes/Heterocycles /aug_0_1057.jpg"

classifier = VisionClassifierInference(
    feature_extractor=ViTFeatureExtractor.from_pretrained(feature_extractor_path),
    model=ViTForImageClassification.from_pretrained(model_path)
)

label = classifier.predict(img_path=img)
print("Predicted class:", label)


Model loaded!
Predicted class: Heterocycles 


In [ ]:
hyp, ref = trainer.evaluate_f1_score()

100%|██████████| 298/298 [00:05<00:00, 50.54it/s]

                                         precision    recall  f1-score   support

                          Heterocycles      0.9277    0.9625    0.9448        80
                            Homocycles      0.9538    0.9118    0.9323        68
                          Hydrocarbons      1.0000    1.0000    1.0000        80
Oxygen and Nitrogen containing acyclic      1.0000    1.0000    1.0000        70

                               accuracy                         0.9698       298
                              macro avg     0.9704    0.9686    0.9693       298
                           weighted avg     0.9701    0.9698    0.9697       298

Logs saved at: ./out/MYKVASIRV2MODEL/2_2023-05-27-10-24-29
